In [1]:
import sys
sys.path.append('../smodels')
from smodels.tools import runtime
#Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
#runtime.modelFile = 'mssmQNumbers.slha'

from smodels.theory import decomposer
from smodels.tools.physicsUnits import fb, GeV, TeV
from smodels.theory.theoryPrediction import theoryPredictionsFor
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.tools.smodelsLogging import setLogLevel
from smodels.particlesLoader import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.theory.model import Model
setLogLevel("info")

In [2]:
# Set the path to the database
database = Database("https://smodels.github.io/database/official210")

INFO in databaseObj.loadBinaryFile() in 483: loading binary db file /home/lessa/.cache/smodels/official210.pcl format version 214
INFO in databaseObj.loadBinaryFile() in 490: Loaded database from /home/lessa/.cache/smodels/official210.pcl in 0.8 secs.


In [3]:
model = Model(BSMparticles=BSMList, SMparticles=SMList)
slhafile = '../EWino/data/slha_scanRandom/ew_qfk25spm.slha'
model.updateParticles(inputFile=slhafile)


# Set main options for decomposition
sigmacut = 0.005*fb
mingap = 5.*GeV

# Decompose model
toplist = decomposer.decompose(model, sigmacut, doCompress=True, doInvisible=True, minmassgap=mingap)


INFO in model.updateParticles() in 385: Loaded 62 BSM particles


In [9]:
elList = sorted(toplist.getElements(), key = lambda el:el.weight.getMaxXsec(), reverse=True)
for el in elList:
    print(el,el.oddParticles,el.weight.getMaxXsec())
    print(el.mass,'\n')

[[[W+]],[[Z]]] [[C1+, N1], [N2, N1]] 2.44E-01 [pb]
[[1.82E+02 [GeV], 5.15E+01 [GeV]], [1.87E+02 [GeV], 5.15E+01 [GeV]]] 

[[[W-]],[[W+]]] [[C1-, N1~], [C1+, N1]] 2.39E-01 [pb]
[[1.82E+02 [GeV], 5.15E+01 [GeV]], [1.82E+02 [GeV], 5.15E+01 [GeV]]] 

[[[W-]],[[Z]]] [[C1-, N1~], [N2, N1]] 1.35E-01 [pb]
[[1.82E+02 [GeV], 5.15E+01 [GeV]], [1.87E+02 [GeV], 5.15E+01 [GeV]]] 

[[[W+]],[[Z]]] [[C1+, N1], [N3, N1]] 1.30E-01 [pb]
[[1.82E+02 [GeV], 5.15E+01 [GeV]], [1.88E+02 [GeV], 5.15E+01 [GeV]]] 

[[[W+]],[[higgs]]] [[C1+, N1], [N3, N1]] 1.08E-01 [pb]
[[1.82E+02 [GeV], 5.15E+01 [GeV]], [1.88E+02 [GeV], 5.15E+01 [GeV]]] 

[[[Z]],[[Z]]] [[N2, N1], [N3, N1]] 1.04E-01 [pb]
[[1.87E+02 [GeV], 5.15E+01 [GeV]], [1.88E+02 [GeV], 5.15E+01 [GeV]]] 

[[[Z]],[[higgs]]] [[N2, N1], [N3, N1]] 8.66E-02 [pb]
[[1.87E+02 [GeV], 5.15E+01 [GeV]], [1.88E+02 [GeV], 5.15E+01 [GeV]]] 

[[[W-]],[[Z]]] [[C1-, N1~], [N3, N1]] 7.22E-02 [pb]
[[1.82E+02 [GeV], 5.15E+01 [GeV]], [1.88E+02 [GeV], 5.15E+01 [GeV]]] 

[[],[[W+]]] [[N

In [6]:
el.weight.getMaxXsec()

2.93E-02 [pb]